# GCN

Node Property Prediction

In this section we will build our first graph neural network using PyTorch Geometric. Then we will apply it to the task of node property prediction (node classification).

Specifically, we will use GCN as the foundation for your graph neural network ([Kipf et al. (2017)](https://arxiv.org/pdf/1609.02907.pdf)). To do so, we will work with PyG's built-in `GCNConv` layer. 

## Setup

In [1]:
import os
import torch
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.11.0


## Load and Preprocess the Dataset

In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  dataset_name = 'ogbn-arxiv'
  dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
  data = dataset[0]

  # Make the adjacency matrix to symmetric
  data.adj_t = data.adj_t.to_symmetric()

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # If you use GPU, the device should be cuda
  print('Device: {}'.format(device))

  data = data.to(device)
  split_idx = dataset.get_idx_split()
  train_idx = split_idx['train'].to(device)

Device: cuda


## GCN Model

Now we will implement our GCN model!

Please follow the figure below to implement the `forward` function.

![img](../../images/cs224w-colab2-3.png)

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # TODO: Implement this function that initializes self.convs, 
        # self.bns, and self.softmax.

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = None

        # A list of 1D batch normalization layers
        self.bns = None

        # The log softmax layer
        self.softmax = None

        ############# Your code here ############
        ## Note:
        ## 1. You should use torch.nn.ModuleList for self.convs and self.bns
        ## 2. self.convs has num_layers GCNConv layers
        ## 3. self.bns has num_layers - 1 BatchNorm1d layers
        ## 4. You should use torch.nn.LogSoftmax for self.softmax
        ## 5. The parameters you can set for GCNConv include 'in_channels' and 
        ## 'out_channels'. More information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. The only parameter you need to set for BatchNorm1d is 'num_features'
        ## More information please refer to the documentation: 
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        ## (~10 lines of code)
        
        # 1. use torch.nn.ModuleList for self.convs
        # 2. self.convs has num_layers GCNConv layers
        # 5. use 'in_channels' and 'out_channels'
        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim) for i in range(num_layers-2)] + 
            [GCNConv(in_channels=hidden_dim, out_channels=output_dim)]
            )
        
        # 1. use torch.nn.ModuleList for self.bns
        # 3. self.bns has num_layers - 1 BatchNorm1d layers
        # 6. The only parameter you need to set for BatchNorm1d is 'num_features'
        self.bns = torch.nn.ModuleList(
            [torch.nn.BatchNorm1d(num_features=hidden_dim) for i in range(num_layers-1)]
            )

        # 4. use torch.nn.LogSoftmax for self.softmax
        self.softmax = torch.nn.LogSoftmax()
        
        #########################################

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        # TODO: Implement this function that takes the feature tensor x,
        # edge_index tensor adj_t and returns the output tensor as
        # shown in the figure.

        out = None

        ############# Your code here ############
        ## Note:
        ## 1. Construct the network as showing in the figure
        ## 2. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## More information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Don't forget to set F.dropout training to self.training
        ## 4. If return_embeds is True, then skip the last softmax layer
        ## (~7 lines of code)
        
        # 2. use torch.nn.functional.relu and torch.nn.functional.dropout
        # 3. set F.dropout training to self.training
        # from the first layer to n-1 layer
        for conv, bn in zip(self.convs[:-1], self.bns):
            x_f = torch.nn.functional.relu(bn(conv(x, adj_t)))
            if self.training:
                x_f = torch.nn.functional.dropout(x_f, p=self.dropout)
            x = x_f
        
        # 4. If return_embeds is True, then skip the last softmax layer
        # the last layer
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        #########################################

        return out

In [4]:
def train(model, data, train_idx, optimizer, loss_fn):
    # TODO: Implement this function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slicing the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)

    # 1. Zero grad the optimizer
    optimizer.zero_grad()

    # 2. Feed the data into the model
    out = model(data.x, data.adj_t)

    # 3. Slicing the model output and label by train_idx
    sliced_output, sliced_label = out[train_idx], data.y[train_idx].reshape(-1)

    # 4. Feed the sliced output and label to loss_fn
    loss = loss_fn(sliced_output, sliced_label)

    #########################################

    loss.backward()
    optimizer.step()

    return loss.item()

In [5]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # TODO: Implement this function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here

    out = model(data.x, data.adj_t)
    
    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [6]:
# Please do not change the args
if 'IS_GRADESCOPE_ENV' not in os.environ:
  args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
  }
  args

In [7]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [8]:
import copy
if 'IS_GRADESCOPE_ENV' not in os.environ:
  # reset the parameters to initial random value
  model.reset_parameters()

  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
  loss_fn = F.nll_loss

  best_model = None
  best_valid_acc = 0

  for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

C:\Users\user\anaconda3\envs\gnn\lib\site-packages\ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 01, Loss: 4.0346, Train: 29.90%, Valid: 32.52% Test: 30.71%
Epoch: 02, Loss: 2.3318, Train: 24.67%, Valid: 22.16% Test: 27.45%
Epoch: 03, Loss: 1.9201, Train: 32.05%, Valid: 27.90% Test: 32.16%
Epoch: 04, Loss: 1.7597, Train: 37.44%, Valid: 34.33% Test: 35.43%
Epoch: 05, Loss: 1.6729, Train: 34.51%, Valid: 30.50% Test: 27.29%
Epoch: 06, Loss: 1.5858, Train: 35.09%, Valid: 32.65% Test: 29.51%
Epoch: 07, Loss: 1.5176, Train: 36.10%, Valid: 34.45% Test: 31.95%
Epoch: 08, Loss: 1.4656, Train: 37.75%, Valid: 36.59% Test: 35.23%
Epoch: 09, Loss: 1.4186, Train: 39.37%, Valid: 39.08% Test: 39.62%
Epoch: 10, Loss: 1.3821, Train: 40.02%, Valid: 37.46% Test: 39.68%
Epoch: 11, Loss: 1.3509, Train: 39.30%, Valid: 33.63% Test: 35.36%
Epoch: 12, Loss: 1.3273, Train: 39.09%, Valid: 31.49% Test: 32.18%
Epoch: 13, Loss: 1.3065, Train: 41.37%, Valid: 34.63% Test: 35.20%
Epoch: 14, Loss: 1.2826, Train: 45.66%, Valid: 41.70% Test: 43.19%
Epoch: 15, Loss: 1.2578, Train: 49.50%, Valid: 48.52% Test: 50

In [9]:
best_result = test(best_model, data, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 73.93%, Valid: 71.98% Test: 70.87%


C:\Users\user\anaconda3\envs\gnn\lib\site-packages\ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## references

cs224w colab 2
https://colab.research.google.com/drive/1BRPw3WQjP8ANSFz-4Z1ldtNt9g7zm-bv?usp=sharing